In [1]:



 #-*- coding: utf-8 -*-
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import os, sys
lib_path = os.path.abspath(os.path.join(os.__file__, '..','lib'))
 ## 원하는 lib이 있는 경로 지정
sys.path.append(lib_path)
import sonya_lib

ModuleNotFoundError: No module named 'sonya_lib'

In [2]:
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
# from pyimagesearch import datasets
# from pyimagesearch import models

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import numpy as np
import argparse
import locale
import os
import sys


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')


import matplotlib
import matplotlib.pyplot as plt

# %matplotlib inline

import keras
import tensorflow as tf

# -------------------------
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import auc
from sklearn.metrics import roc_curve # plot_roc_curve
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical


Using TensorFlow backend.


In [ ]:
path = './total_836_200622_test.xlsx'
df = pd.read_excel(path)


def homogeneous(row):
    if row['Enhancement'] == 'no-enhancement':
        return 0.25
    elif row['Enhancement'] == 'weak':
        return 0.50
    elif row['Enhancement'] == 'moderate':
        return 0.75
    elif row['Enhancement'] == 'high':
        return 1
    else:
        return 0

def normalize(dataset):
    dataNorm = ((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataset = dataNorm
    return dataset


df_filtered = df[[u'시행시나이','cT']]


# df_filtered = df[[u'시행시나이','cT','aAverage','aSD','cAverage','cSD']]
# df_filtered['cT'] = normalize(df['cT'])
# df_filtered['cT'] = normalize(df['cT'])
# df_filtered['L/R'] = np.where(df['L/R'] == 'L', 1, 0) # L/R
df_filtered['hu_diff'] = normalize(df['aAverage'] - df['cAverage'])
df_filtered['homogeneous'] = df.apply(homogeneous, axis=1)
df_filtered['hetero'] = np.where(df['Enhancement'] == 'hetero', 1, 0)
df_filtered['rim'] = np.where(df['Enhancement'] == 'rim', 1, 0)
df_filtered['clustered'] = np.where(df['Enhancement'] == 'clustered', 1, 0)
df_filtered['non-mass'] = np.where(df['Enhancement'] == 'non-mass', 1, 0)
df_filtered['label'] = df['pN_modify']

df_filtered = df_filtered.rename(columns={u'시행시나이': u'age'})

properties = list(df_filtered.columns.values)
properties.remove('label')
X = df_filtered[properties]
y = df_filtered['label']
num_input = len(X.columns)




In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
print(kfold)
accuracy = []


for train, validation in kfold.split(X, y):

    X_train, X_test = X.iloc[train], X.iloc[validation]
    y_train, y_test = y.iloc[train], y.iloc[validation]

    model = Sequential()

    model.add(Dense(16, activation='relu', input_dim=num_input))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 학습 데이터를 이용해서 학습
    hsit = model.fit(X_train, y_train, epochs=1000, batch_size=5)

    # roc curve
    fpr, tpr, thresholds = roc_curve()

    # 테스트 데이터를 이용해서 검증
    k_accuracy = '%.4f' % (model.evaluate(X_test, y_test)[1])
    accuracy.append(k_accuracy)

# 전체 검증 결과 출력


In [ ]:
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation mean Accuracy: {}'.format(np.mean(accuracy)))
